In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:

import dolfinx as dfx

from dolfinx.fem.petsc import NonlinearProblem

%matplotlib widget
from matplotlib import pyplot as plt
# plt.style.use('fivethirtyeight')

from mpi4py import MPI

import numpy as np

from pathlib import Path

from petsc4py import PETSc

import ufl

from pyMoBiMP.cahn_hilliard_utils import (
    cahn_hilliard_form,
    charge_discharge_stop, 
    AnalyzeOCP,
    y_of_c,
    c_of_y,
    populate_initial_data)

from pyMoBiMP.fenicsx_utils import (evaluation_points_and_cells,
                           get_mesh_spacing,
                           time_stepping,
                           NewtonSolver,
                           Fenicx1DOutput,
                           VTXOutput)

from pyMoBiMP.gmsh_utils import dfx_spherical_mesh

from pyMoBiMP.plotting_utils import (
    add_arrow, 
    plot_charging_cycle, 
    plot_time_sequence,
    animate_time_series)

comm_world = MPI.COMM_WORLD

In [ ]:
# Discretization
# --------------

# Set up the mesh
n_elem = 128

mesh = dfx.mesh.create_unit_interval(comm_world, n_elem)

dx_cell = get_mesh_spacing(mesh)

print(f"Cell spacing: h = {dx_cell}")

# For later plotting use
x = np.linspace(0, 1, 101)
points_on_proc, cells = evaluation_points_and_cells(mesh, x)

T = 2.  # ending time

# Initial timestep size
dt = dfx.fem.Constant(mesh, dx_cell * 0.01)

In [ ]:
elem1 = ufl.FiniteElement("Lagrange", mesh.ufl_cell(), 1)

V = dfx.fem.FunctionSpace(mesh, elem1 * elem1)  # A mixed two-component function space

In [ ]:
# The mixed-element functions
u = dfx.fem.Function(V)
u0 = dfx.fem.Function(V)

In [ ]:
# Compute the chemical potential df/dc

free_energy = lambda c, log: c * log(c) + (1 - c) * log(1 - c)

fig, ax = plt.subplots()

eps = 1e-3

c_plot = np.linspace(eps, 1-eps, 200)

ax.plot(c_plot, free_energy(c_plot, np.log))

ax.set_xlabel(r"$c$")
ax.set_ylabel(r"$F(c)$")

plt.show()

In [ ]:
# Experimental setup
# ------------------

# charging current
I_charge = dfx.fem.Constant(mesh, 1.0)

def experiment(t, u, I_charge):

    return charge_discharge_stop(t, u, I_charge, c_of_y=lambda y: c_of_y(y),
                                 stop_on_full=False, stop_at_empty=False, cycling=False)

event_params = dict(I_charge=I_charge)

In [ ]:
# The variational form
# --------------------

F = cahn_hilliard_form(
    u,
    u0,
    dt,
    free_energy=lambda c: free_energy(c, ufl.ln),
    theta=0.75,
    c_of_y=lambda y: c_of_y(y),
    M=lambda c: 1. * c * (1 - c),
    lam=0.00001,
    **event_params
)

In [ ]:
# Initial data
# ------------

u_ini = dfx.fem.Function(V)

# Constant
c_ini_fun = lambda x: eps * np.ones_like(x[0])

# Initial charge distribution.
# c_ini_fun = lambda x: eps + 0.5 * np.sin(np.pi * x[0])

populate_initial_data(u_ini, c_ini_fun, lambda c: free_energy(c, ufl.ln))

W, _ = V.sub(0).collapse()

c = dfx.fem.Function(W)
c.interpolate(dfx.fem.Expression(c_of_y(u_ini.sub(0)), W.element.interpolation_points()))

plt.figure()

plt.plot(x, u_ini.sub(0).eval(points_on_proc, cells), label="y")
plt.plot(x, c.eval(points_on_proc, cells), label="c")
plt.plot(x, u_ini.sub(1).eval(points_on_proc, cells), label=r"$\mu$")

plt.legend()

plt.show()

In [ ]:
problem = NonlinearProblem(F, u)

solver = NewtonSolver(comm_world, problem)

In [ ]:
u.interpolate(u_ini)

n_out = 51

output_np = Fenicx1DOutput(u, np.linspace(0, T, n_out), x)

rt_analysis = AnalyzeOCP(c_of_y = lambda y: c_of_y(y))

time_stepping(
    solver,
    u,
    u0,
    T,
    dt,
    dt_increase=1.01,
    dt_max=1e-3,
    event_handler=experiment,
    output=(output_np),
    runtime_analysis=rt_analysis,
    **event_params,
)

In [ ]:
fig, ax = plot_time_sequence(output_np, lambda y: np.exp(y) / (1 + np.exp(y)))

plt.show()

In [ ]:
from plotting_utils import PyvistaAnimation

anim = PyvistaAnimation(
    output_np,
    c_of_y=lambda y: np.exp(y) / (np.exp(y) + 1),
    res=0.1,
    clim=[0.0, 1.0],
    cmap="hot",)

widget = anim.get_slider_widget()

anim.write_vtk_output("results/lin_diff.vtk")

In [ ]:
# anim_gif = PyvistaAnimation(
#     output,
#     c_of_y=lambda y: c_of_y(y, np.exp),
#     res=0.1,
#     clim=[0.0, 1.0],
#     cmap="hot",)

# results_folder = Path("results")
# results_folder.mkdir(exist_ok=True, parents=True)

# filename = results_folder / "linear_diffusion.gif"

# anim_gif.get_gif_animation(filename)

In [ ]:
fig, ax = plt.subplots()

c = c_of_y(u.sub(0))
mu = u.sub(1).eval(points_on_proc, cells)

W = V.sub(1).collapse()[0]

c = ufl.variable(c)
dFdc = ufl.diff(free_energy(c, ufl.ln), c)

chem_pot = dfx.fem.Function(W)
chem_pot.interpolate(dfx.fem.Expression(dFdc, W.element.interpolation_points()))
chem_pot = chem_pot.eval(points_on_proc, cells)

ax.plot(x, mu)
ax.plot(x, chem_pot)

plt.show()

In [ ]:
q, f_bar, mu_bc = np.array(rt_analysis.data).T
t = rt_analysis.t

fig, ax = plt.subplots()

ax1 = ax.twinx()

(p1,) = ax1.plot(t, q, '--', label=r"$q$")
ax1.set_ylabel(r"$q$")

(p2,) = ax.plot(t, mu_bc, label=r"$\left. \mu \right|_{\partial \omega_I}$")
(p3,) = ax.plot(t, f_bar, label=r"$\bar f$")

labs = [l.get_label() for l in [p1, p2, p3]]

ax.legend([p1, p2, p3], labs, loc="lower right")

plt.show()

In [ ]:
q, f_bar, mu_bc = np.array(rt_analysis.data).T

eps = 1e-3

a = 6/4
b = 0.2
c = 5.0

free_energy = lambda q: np.log(q / (1 - q))
chart = plot_charging_cycle([(I_charge.value, q, mu_bc)], lambda q: free_energy(q))